## 1. Setup Environment

Install required dependencies (run once per session)

In [ ]:
# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running on Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA version: {torch.version.cuda}")
else:
    print("No GPU detected - will use CPU (slower)")

In [ ]:
%%capture
!pip install -q paddleocr opencv-python pdf2docx python-docx PyMuPDF
!pip install -q numpy==1.26.4 Pillow torch transformers vietocr
!pip install -q craft-text-detector matplotlib opencv-contrib-python torchvision

## 2. Clone Repository (Colab only)

If running on Colab, clone the repository to access the modules

In [ ]:
if IN_COLAB:
    # Clone repository
    !git clone https://github.com/HuyTran28/23CLCT2_TraditionalMedicineChatbot.git
    
    # Change to OCR directory
    import os
    os.chdir('/content/23CLCT2_TraditionalMedicineChatbot/ocr')
    print("Repository cloned and working directory set")
else:
    print("Skipping - not on Colab")

## 3. Upload Input Files (Colab only)

Upload your PDF files to process

In [ ]:
if IN_COLAB:
    from google.colab import files
    from pathlib import Path
    
    # Create input directory
    Path('./input').mkdir(exist_ok=True)
    
    print("Upload your PDF files:")
    uploaded = files.upload()
    
    # Move uploaded files to input directory
    for filename in uploaded.keys():
        import shutil
        shutil.move(filename, f'./input/{filename}')
    
    print(f"\nUploaded {len(uploaded)} file(s) to ./input/")
else:
    print("Skipping - not on Colab")
    print("   Place your PDF files in the 'input' folder")

## 4. Initialize Pipeline

Set up the OCR pipeline with your preferred settings

In [ ]:
import sys
import logging
from pathlib import Path

# Add modules to path
sys.path.insert(0, './modules')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

from modules.pipeline import OCRPipeline

print("Modules imported successfully")

In [ ]:
# Configuration
CONFIG = {
    'input_dir': './input',           # Input folder containing PDFs
    'output_dir': './output',         # Output folder for Word files
    'temp_dir': './temp',             # Temporary files directory
    'dpi': 300,                       # Resolution for image conversion
    'enable_preprocessing': True,     # Enable image enhancement
    'auto_detect': True,              # Auto-detect digital vs scanned
}

# Initialize pipeline
pipeline = OCRPipeline(
    output_dir=CONFIG['output_dir'],
    temp_dir=CONFIG['temp_dir'],
    dpi=CONFIG['dpi'],
    enable_preprocessing=CONFIG['enable_preprocessing'],
    auto_detect=CONFIG['auto_detect']
)

print("Pipeline initialized with configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 5. Process Single File

Process a single PDF file

In [ ]:
# Specify the PDF file to process
pdf_file = "sample.pdf"  # Change this to your filename

input_path = Path(CONFIG['input_dir']) / pdf_file

if not input_path.exists():
    print(f"File not found: {input_path}")
    print(f"\nAvailable files in {CONFIG['input_dir']}:")
    for f in Path(CONFIG['input_dir']).glob('*.pdf'):
        print(f"  - {f.name}")
else:
    print(f"Processing: {pdf_file}")
    print("=" * 60)
    
    try:
        output_path = pipeline.process_pdf(input_path, mode=None)
        print("\n" + "=" * 60)
        print(f"Success! Output saved to: {output_path}")
    except Exception as e:
        print(f"\nError: {e}")
        import traceback
        traceback.print_exc()

## 6. Batch Process All Files

Process all PDF files in the input directory

In [ ]:
print(f"Batch processing PDFs from: {CONFIG['input_dir']}")
print("=" * 60)

try:
    results = pipeline.process_batch(
        input_dir=CONFIG['input_dir'],
        pattern="*.pdf",
        mode=None  # Auto-detect mode
    )
    
    # Print summary
    print("\n" + "=" * 60)
    print("PROCESSING SUMMARY")
    print("=" * 60)
    
    success_count = 0
    for r in results:
        status_icon = "✓" if r["status"] == "success" else "✗"
        print(f"{status_icon} {Path(r['input']).name}")
        
        if r["status"] == "success":
            print(f"   → {r['output']}")
            success_count += 1
        else:
            print(f"   Error: {r.get('error', 'Unknown error')}")
    
    print("\n" + "=" * 60)
    print(f"Completed: {success_count}/{len(results)} files processed successfully")
    print("=" * 60)
    
except Exception as e:
    print(f"\nBatch processing error: {e}")
    import traceback
    traceback.print_exc()

## 7. Download Results (Colab only)

Download the converted Word files

In [ ]:
if IN_COLAB:
    from google.colab import files
    import zipfile
    from pathlib import Path
    
    output_dir = Path(CONFIG['output_dir'])
    docx_files = list(output_dir.glob('*.docx'))
    
    if not docx_files:
        print("No output files found")
    elif len(docx_files) == 1:
        # Download single file
        print(f"Downloading: {docx_files[0].name}")
        files.download(str(docx_files[0]))
    else:
        # Zip and download multiple files
        zip_path = 'ocr_results.zip'
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for docx_file in docx_files:
                zipf.write(docx_file, docx_file.name)
        
        print(f"Downloading {len(docx_files)} files as {zip_path}")
        files.download(zip_path)
else:
    print("Skipping - not on Colab")
    print(f"   Output files are in: {CONFIG['output_dir']}")

## 8. Cleanup (Optional)

Remove temporary files to free up space

In [ ]:
import shutil
from pathlib import Path

temp_dir = Path(CONFIG['temp_dir'])
if temp_dir.exists():
    shutil.rmtree(temp_dir)
    print(f"Cleaned up temporary files in {temp_dir}")
else:
    print("No temporary files to clean")